In [1]:
import pandas as pd

In [2]:
signals = pd.read_parquet("signals.parquet.gzip")
metatags = pd.read_parquet("metatags.parquet.gzip")

In [3]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [4]:
articles = intersection(metatags.src.unique(), signals.src.unique() )
signals = signals[signals.src.isin(articles)]
metatags = metatags[metatags.src.isin(articles)]
len(articles)

971

In [5]:
metatags

,src,keywords,themes,summary,origin,url
0,ea74b217d36ec570bb432fb5aa679090,"[AI panic, classrooms, education system, adapt...","[Impact of AI in Education, Adapting to AI in ...",The introduction of AI in classrooms has spark...,20230416,b'https://www.oneusefulthing.org/p/the-future-...
1,841546c0efc4c82f0aabc545a47a09e1,"[backdoor, cybersecurity researchers, encrypte...","[cybersecurity, encryption, vulnerabilities]",A group of cybersecurity researchers has disco...,20230810,b'https://www.vice.com/en/article/4a3n3j/backd...
2,f6637d1fa0e3fe0e1e94bea8b3338ef9,"[Cybersecurity Researchers, Sandbox Satellite,...","[Satellite security, Hack-a-Sat competition, C...",Researchers have launched a satellite called M...,20230708,b'https://spaceref.com/newspace-and-tech/cyber...
3,58612e45effab1e47df9a86b14dfae85,"[regulating artificial intelligence, safeguard...","[Regulating AI, Ethical concerns, Technologica...",The text discusses the challenges of regulatin...,20231209,b'https://thegeneralist.substack.com/p/life-in...
4,7ff6239e13a43e856fa36887902c5ede,"[Chat-GPT, Python, Knowledge Graph, Neo4j, art...","[Artificial Intelligence, Natural Language Pro...",This article discusses how to use Chat-GPT and...,20290911,b'https://towardsdatascience.com/how-to-use-ch...
...,...,...,...,...,...,...
1183,4d1abdf7e702b559c6ccff847ce4d8d0,"[backdooring, summarizerbot, shape opinion, mo...","[Machine learning, Cybersecurity, Manipulation]",This text discusses the risks associated with ...,20221031,b'https://pluralistic.net/2022/10/21/let-me-su...
1184,edbb7336dfb26098bcc966ca17c074b9,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...
1185,3fbb11d6e949d1e662aa6a146bb6cda0,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...
1187,b79a2baa87b68283198416791b93bce4,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...


In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [7]:
#sentences = list(metatags.summary)
sentences = list(metatags.keywords)
sentences = [", ".join(x) for x in sentences]

In [8]:
from sentence_transformers import util


In [9]:
embeddings = model.encode(sentences, convert_to_tensor=True,show_progress_bar=True)
cosine_scores = util.cos_sim(embeddings, embeddings)
#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)
for pair in pairs[0:10]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], pair['score']))

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

trial, digital access, FT.com, Standard Digital, Premium Digital, global news, analysis, expert opinion, Lex, business themes 		 trial, digital access, FT.com, Standard Digital, Premium Digital, global news, analysis, expert opinion, Lex, business themes 		 Score: 1.0000
keyword0 for keyword in top_keywords 		 keyword0 for keyword in top_keywords 		 Score: 1.0000
connected, agile, workloads, competitive, urgency, tactics, culture, chronically overwhelmed, reactive, time management 		 connected, agile, workloads, competitive, urgency, tactics, culture, chronically overwhelmed, reactive, time management 		 Score: 1.0000
counterfeit people, artificial intelligence, immoral act, vandalism, counterfeiting, trust, human freedom, creation, deepfakes, counterfeit digital people 		 counterfeit people, artificial intelligence, immoral act, vandalism, counterfeiting, trust, digital environments, dangerous artifacts, economies, human freedom 		 Score: 0.9684
trial, digital access, FT.com, Standard

It takes 4 mins for 1000 articles. Approx.

In [10]:
dfSent = pd.DataFrame([sentences, list(metatags.src)]).T
dfSent.columns = ["txt","src"]
dfSent.to_parquet("pages/dfSentences.parquet.gzip",compression="gzip")
dfSent

,txt,src
0,"AI panic, classrooms, education system, adapt,...",ea74b217d36ec570bb432fb5aa679090
1,"backdoor, cybersecurity researchers, encrypted...",841546c0efc4c82f0aabc545a47a09e1
2,"Cybersecurity Researchers, Sandbox Satellite, ...",f6637d1fa0e3fe0e1e94bea8b3338ef9
3,"regulating artificial intelligence, safeguards...",58612e45effab1e47df9a86b14dfae85
4,"Chat-GPT, Python, Knowledge Graph, Neo4j, arti...",7ff6239e13a43e856fa36887902c5ede
...,...,...
966,"backdooring, summarizerbot, shape opinion, mod...",4d1abdf7e702b559c6ccff847ce4d8d0
967,"hanging out, book, Sheila Liming, crisis, isol...",edbb7336dfb26098bcc966ca17c074b9
968,"insomniacs, sleeping assistants, Taobao, Chine...",3fbb11d6e949d1e662aa6a146bb6cda0
969,"U.S. government, Nvidia, high-performance comp...",b79a2baa87b68283198416791b93bce4


In [11]:
dfPairs = pd.DataFrame(pairs)
dfPairs["score"] = dfPairs["score"].astype(float)
dfPairs["i"] = dfPairs["index"].apply(lambda x: x[0])
dfPairs["j"] = dfPairs["index"].apply(lambda x: x[1])
dfPairs.to_parquet("pages/dfPairs.parquet.gzip",compression="gzip")
dfPairs

,index,score,i,j
0,"[33, 963]",1.000000,33,963
1,"[365, 880]",1.000000,365,880
2,"[692, 763]",1.000000,692,763
3,"[106, 322]",0.968448,106,322
4,"[33, 411]",0.967408,33,411
...,...,...,...,...
470930,"[105, 875]",-0.040864,105,875
470931,"[271, 275]",-0.040947,271,275
470932,"[338, 858]",-0.050092,338,858
470933,"[436, 858]",-0.056254,436,858


In [12]:
def getClosest(i):
    RES = dfPairs[ ((dfPairs["i"] == i) & (dfPairs["j"] != i)) \
      + ((dfPairs["j"] == i) & (dfPairs["i"] != i))]

    closest = list(RES.sort_values(by="score", ascending=False )["j"][:10])
    return [metatags.iloc[x].src for x in closest]

In [13]:
i = 3
RES = dfPairs[ ((dfPairs["i"] == i) & (dfPairs["j"] != i)) ]

closest = list(RES.sort_values(by="score", ascending=False )["j"][:10])
closest

/tmp/ipykernel_1825644/1472497355.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  closest = list(RES.sort_values(by="score", ascending=False )["j"][:10])


[799, 920, 134, 512, 796, 249, 42, 354, 500, 333]

In [14]:
metatags = metatags.reset_index()
metatags = metatags.drop("index",axis=1)
metatags

,src,keywords,themes,summary,origin,url
0,ea74b217d36ec570bb432fb5aa679090,"[AI panic, classrooms, education system, adapt...","[Impact of AI in Education, Adapting to AI in ...",The introduction of AI in classrooms has spark...,20230416,b'https://www.oneusefulthing.org/p/the-future-...
1,841546c0efc4c82f0aabc545a47a09e1,"[backdoor, cybersecurity researchers, encrypte...","[cybersecurity, encryption, vulnerabilities]",A group of cybersecurity researchers has disco...,20230810,b'https://www.vice.com/en/article/4a3n3j/backd...
2,f6637d1fa0e3fe0e1e94bea8b3338ef9,"[Cybersecurity Researchers, Sandbox Satellite,...","[Satellite security, Hack-a-Sat competition, C...",Researchers have launched a satellite called M...,20230708,b'https://spaceref.com/newspace-and-tech/cyber...
3,58612e45effab1e47df9a86b14dfae85,"[regulating artificial intelligence, safeguard...","[Regulating AI, Ethical concerns, Technologica...",The text discusses the challenges of regulatin...,20231209,b'https://thegeneralist.substack.com/p/life-in...
4,7ff6239e13a43e856fa36887902c5ede,"[Chat-GPT, Python, Knowledge Graph, Neo4j, art...","[Artificial Intelligence, Natural Language Pro...",This article discusses how to use Chat-GPT and...,20290911,b'https://towardsdatascience.com/how-to-use-ch...
...,...,...,...,...,...,...
966,4d1abdf7e702b559c6ccff847ce4d8d0,"[backdooring, summarizerbot, shape opinion, mo...","[Machine learning, Cybersecurity, Manipulation]",This text discusses the risks associated with ...,20221031,b'https://pluralistic.net/2022/10/21/let-me-su...
967,edbb7336dfb26098bcc966ca17c074b9,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...
968,3fbb11d6e949d1e662aa6a146bb6cda0,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...
969,b79a2baa87b68283198416791b93bce4,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...


In [15]:
metatags["closest"] = ""
for ix, row in metatags.iterrows():
    row["closest"] = getClosest(ix)
metatags

/tmp/ipykernel_1825644/2972742187.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  closest = list(RES.sort_values(by="score", ascending=False )["j"][:10])


,src,keywords,themes,summary,origin,url,closest
0,ea74b217d36ec570bb432fb5aa679090,"[AI panic, classrooms, education system, adapt...","[Impact of AI in Education, Adapting to AI in ...",The introduction of AI in classrooms has spark...,20230416,b'https://www.oneusefulthing.org/p/the-future-...,"[b30a4282af9e53ca673438a8223d9525, 5a06b540b77..."
1,841546c0efc4c82f0aabc545a47a09e1,"[backdoor, cybersecurity researchers, encrypte...","[cybersecurity, encryption, vulnerabilities]",A group of cybersecurity researchers has disco...,20230810,b'https://www.vice.com/en/article/4a3n3j/backd...,"[f6637d1fa0e3fe0e1e94bea8b3338ef9, ee0bcad312f..."
2,f6637d1fa0e3fe0e1e94bea8b3338ef9,"[Cybersecurity Researchers, Sandbox Satellite,...","[Satellite security, Hack-a-Sat competition, C...",Researchers have launched a satellite called M...,20230708,b'https://spaceref.com/newspace-and-tech/cyber...,"[ae919b08f9b58de655abd2fea03c1169, 0b6d917158c..."
3,58612e45effab1e47df9a86b14dfae85,"[regulating artificial intelligence, safeguard...","[Regulating AI, Ethical concerns, Technologica...",The text discusses the challenges of regulatin...,20231209,b'https://thegeneralist.substack.com/p/life-in...,"[2cfdc6fba6f11e089c2df3cda1604174, df59e2cf338..."
4,7ff6239e13a43e856fa36887902c5ede,"[Chat-GPT, Python, Knowledge Graph, Neo4j, art...","[Artificial Intelligence, Natural Language Pro...",This article discusses how to use Chat-GPT and...,20290911,b'https://towardsdatascience.com/how-to-use-ch...,"[e0f6ae75e034df0a32044fe8a9075673, f43094a456d..."
...,...,...,...,...,...,...,...
966,4d1abdf7e702b559c6ccff847ce4d8d0,"[backdooring, summarizerbot, shape opinion, mo...","[Machine learning, Cybersecurity, Manipulation]",This text discusses the risks associated with ...,20221031,b'https://pluralistic.net/2022/10/21/let-me-su...,"[4d1abdf7e702b559c6ccff847ce4d8d0, 4d1abdf7e70..."
967,edbb7336dfb26098bcc966ca17c074b9,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...,"[edbb7336dfb26098bcc966ca17c074b9, edbb7336dfb..."
968,3fbb11d6e949d1e662aa6a146bb6cda0,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...,"[3fbb11d6e949d1e662aa6a146bb6cda0, 3fbb11d6e94..."
969,b79a2baa87b68283198416791b93bce4,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...,"[b79a2baa87b68283198416791b93bce4, b79a2baa87b..."


In [16]:
metatags.to_parquet("pages/metatags_with_closest.parquet.gzip", compression="gzip")

In [17]:
metatags = pd.read_parquet("pages/metatags_with_closest.parquet.gzip")

In [18]:
signals

,Signal,Change,10y horizon,Driving force,src
0,AI's impact on education ...,Adaptation and improvement,"Integration of AI in classrooms, AI tutoring,...",Potential for improved learning and teaching ...,ea74b217d36ec570bb432fb5aa679090
1,AI cheating in classrooms ...,Widespread and undetectable,"Continued AI cheating, advancements in AI tec...",Easier access to AI technology and the desire...,ea74b217d36ec570bb432fb5aa679090
2,AI transforming classrooms ...,Fundamental transformation,"Flipped classrooms, personalized learning, en...",Potential to enhance learning experience and ...,ea74b217d36ec570bb432fb5aa679090
3,Concerns about the impact of calculators on e...,Transition and adaptation,"Calculators integrated into curriculum, chang...",Desire for increased student engagement and p...,ea74b217d36ec570bb432fb5aa679090
4,Potential impact of AI on literacy skills ...,Uncertain,"Potential decline in literacy skills, relianc...",Potential concern for declining writing skill...,ea74b217d36ec570bb432fb5aa679090
...,...,...,...,...,...
6146,Middle East countries impacted by restrictions,Limitation on access to high-performance GPUs,Shift in AI technology supply chains ...,Concerns over technology diversion to China ...,b79a2baa87b68283198416791b93bce4
6147,Saudi Arabia and UAE strengthening AI prowess,Increased focus on AI development,Greater investment and collaboration in AI ...,Desire to advance AI capabilities ...,b79a2baa87b68283198416791b93bce4
6148,U.S. concern over technology diversion to China,Efforts to prevent Chinese AI development,Stricter regulations and controls on technology,Protecting domestic AI industry and capabilit...,b79a2baa87b68283198416791b93bce4
6149,Nvidia working with U.S. government,Collaboration to address licensing issue,Resolution of licensing requirements ...,Maintaining business operations and revenue s...,b79a2baa87b68283198416791b93bce4


In [19]:
titles = pd.read_parquet("titles.parquet.gzip")
titles
TITLES = {}
for ix, row in titles.iterrows():
    TITLES[row["src"]] =  row["title"] 

In [35]:
titles

,src,title
0,ea74b217d36ec570bb432fb5aa679090,The Impact of AI on Education
1,841546c0efc4c82f0aabc545a47a09e1,Researchers Uncover Backdoor in Encrypted Radi...
2,f6637d1fa0e3fe0e1e94bea8b3338ef9,Hack-a-Sat: Researchers Launch Sandbox Satelli...
3,7ff6239e13a43e856fa36887902c5ede,Building a Knowledge Graph with Chat-GPT and P...
4,58c165e1a46408307f2b60f63df7c549,E-seeds Drill Themselves into Soil
...,...,...
1062,edbb7336dfb26098bcc966ca17c074b9,The Radical Power of Hanging Out
1063,3fbb11d6e949d1e662aa6a146bb6cda0,China's Insomniacs Turn to Sleeping Assistants
1064,e8b7faf708568f9f39abe04b778c4631,The Impact of ChatGPT on Efficiency and Inform...
1065,b79a2baa87b68283198416791b93bce4,US Restricts Nvidia GPU Sales to Thwart China'...


In [20]:
metatags["kwsl"] = metatags["keywords"].apply(lambda lst: [x.lower().replace('"','') for x in lst])


In [22]:
metatags[metatags.src == "1ef0f968ac542dcb76eef026bdb6791e"]

,src,keywords,themes,summary,origin,url,closest,kwsl
595,1ef0f968ac542dcb76eef026bdb6791e,"[Generative AI, AI detectors, AI writing, chea...","[Generative AI, AI detection and limitations, ...","In this article, the author discusses several ...",20231203,b'https://www.oneusefulthing.org/p/what-people...,"[e1fbb09ec5e66a8a6d4eff2126eefb40, 1ef0f968ac5...","[generative ai, ai detectors, ai writing, chea..."


In [29]:
for ix, row in metatags.iterrows():
    try:
        ID = row["src"]
        TXT = "# __"+TITLES[ID] +"__, from (["+row.origin+"](https://kghosh.substack.com/p/"+row.origin+").)\n\n"
        TXT += "__[External link]("+str(row.url)[2:-1]+")__\n\n"
        TXT += "\n\n## Summary\n\n"
        TXT += row["summary"] + "\n\n## Keywords\n\n"
        TXT += "* "+"\n* ".join(row["keywords"])
        TXT += "\n\n## Themes\n\n"
        TXT += "* "+"\n* ".join(row["themes"])
        TXT += "\n\n## Signals\n\n"    
        SIG = signals[signals.src == ID]
        TXT += SIG[list(SIG.columns)[:-1]].to_markdown(index=False)
        if 1:
            TXT += "\n\n## Closest\n\n"
            TXT += "* "+"\n* ".join([ "["+TITLES[x]+"]("+x+")" for x in row["closest"]])
        with open("docs/"+ID+".md", "w") as f:
            f.write(TXT) 
    except:
        print("Error with",row["src"])


Error with ea74b217d36ec570bb432fb5aa679090
Error with 58612e45effab1e47df9a86b14dfae85
Error with 9316b88ba7800087e445370bfa1e3943
Error with 9191375fe5b80e2b93a7369a79e63957
Error with 364fecbe1c46f164e48d52c279422351
Error with c24651d7a89dbc792142e26a99579bfe
Error with 4270753a719b34ed63a12b3917635193
Error with 115b59fc3f0d7b148482545adb1a8038
Error with 7eff1fa6b2dda89fa9c1470272891080
Error with e99358609284de847f0d590948f5b265
Error with 1dea025d0138e53b9f644748f63a15bc
Error with 40a7d90f20bae6180001e65049ae853b
Error with 09d755bf91d4fd4e1adf09d7ee1f4656
Error with fab122d29aed97045e0cc1ea77bdef44
Error with 2c79a113d206a8ec8ec147422fcea12c
Error with 5787bdf4f1ad4805a622fc45bf75f023
Error with 6f19a30cab90c95bd01c25851126531e
Error with e9338e8177e66b338acd8753cbe0e3cf
Error with 22d0934658c9fc05be2fd8ded3007cf4
Error with 8b7b3c515859db4c0f6537824019cb7b
Error with 8dbe5c04a6ddbbb51645e5d8e8af3adc
Error with 607bce5ceffd29c4ba8997068ebde091
Error with 47d0fd7d569a766b50f66

Error with c7c9b6f50bfa3280f1f27f83103d2d50
Error with 633655a8e8a99ad217f019928a2b8dc1
Error with 741fe3c74904ed78f07285a0a13f87f7
Error with f4c4b44832de983ad30047e9b2632bc1
Error with 95ddb5a9e335a7f5f977a4ff7b1d17e3
Error with ec9cf2ccb630113e57cfdd18230f129a
Error with b0d1ed951415ca875228752e72f21c58
Error with 4a82d13416aa639187f912c1c8282e0f
Error with bfc9a024f93db97272c074662f73af22
Error with 992602cc0ddc531bb5e3ab280132a31f
Error with 578c4ca8c38df5eb168308242e9ea711
Error with c7cd8debffd00db3905b4d70b56ea40e
Error with affe5112e508a38d53dff8a0642f1eeb
Error with a7a5e8b41b65f1390a0098e577dbea46
Error with ca4e056c63caa99162778073ba814ac1
Error with c41a64e50d0280a765ebdac4f8094446
Error with 29806fd07d41c5c25550b367b80f445c
Error with 14a8b49f5342c0428f8f03db633c57d5
Error with feb0555f9c54012995fdac8ef2410b8e
Error with f9b73418962acd66fb1ae29b21d7f943
Error with 3f93494c0a224e19c9a86b8b566bb280
Error with 5a36304d6c6c3617ca17501078bbc8f9
Error with 550199f663e5e26f2824e

In [30]:
ID

'69637dcd83c48ebde0610a61a27b1989'

In [31]:
allkw = []
for kw in metatags.themes:# keywords: #as an alternative
    allkw = allkw+list(kw)
KW = list(set(allkw))
KW = list(set([x.capitalize().replace('"','') for x in KW]))
KW = [x for x in KW if not x.isnumeric()]
KW.sort()
print(len(KW))
KW

2427


['2024 problem in logistics',
 '3d imaging',
 '3d printing technology in construction',
 'Academic vs. intellectual',
 'Access plans',
 'Access to culture',
 'Access to fresh groceries',
 'Access to tools and resources',
 'Accessibility and applications of ai to everyday life',
 'Accountability',
 'Accounting',
 'Activation',
 'Activism',
 'Activisme écologique',
 'Activity partners',
 'Adaptation of education system',
 'Adapting to ai in the classroom',
 'Adaptive reuse apartments',
 'Adding',
 'Additional revenue',
 'Adhd in the workplace',
 'Advanced bionic limb',
 'Advanced technology',
 'Advancements in ai',
 'Advancements in civil engineering',
 'Advancements in generative ai',
 'Advancements in watch technology',
 'Advances in artificial intelligence',
 'Advances in language models',
 'Advances in medical imaging',
 'Advantages of ai-based animations',
 'Advantages of chatgpt in agile coaching',
 'Advantages of gen z in navigating software',
 'Advantages of magnesium as a struct

In [32]:
import string
LETTERS = string.ascii_uppercase
LETTERS

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [33]:
INDEX = "Index \n\n"

for letter in LETTERS:
    kws = [x for x in KW if x[0] == letter]

    if len(kws):
        INDEX += "\n\n## Letter '"+letter.upper()+"' \n\n"
        for kw in kws:
            INDEX += "\n* __"+kw+"__: "
            mask = metatags.themes.apply(lambda x: any(item for item in [kw] if item in [X.capitalize() for X in x]))
            lst = metatags[mask].src.tolist()
            INDEX += ", ".join([ "["+TITLES[x]+"]("+x+")" for x in lst ])
INDEX += "\n\n## Others\n\n"
kws = [x for x in KW if x[0].isnumeric()]
for kw in kws:
    INDEX += "\n* __"+kw+"__: "
    mask = metatags.kwsl.apply(lambda x: any(item for item in [kw] if item in [X.capitalize() for X in x]))
    lst = metatags[mask].src.tolist()
    INDEX += ", ".join([ "["+TITLES[x]+"]("+x+")" for x in lst ])

KeyError: '45b33e1316e998abe15730c2aeaaa70b'

In [34]:
with open("docs/index.md", "w") as f:
    f.write(INDEX) 

In [ ]:
INDEX